In [1]:
import xgboost as xgb
import sys,random
import pickle
import os
import numpy as np

In [2]:
# import modin.pandas as pd
import pandas as pd

In [3]:
if not os.path.exists('featurescore'):
    os.mkdir('featurescore')
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('preds'):
    os.mkdir('preds')

### 读取有标签的

In [4]:
%%time
# 导入训练数据集
data_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id','loan_hour'])
data_raw = pd.read_csv('../../preprocess_data_new/train_ax_nodup.csv',nrows=33465).drop(columns=['id','loan_dt','tag'])
data_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',nrows=33465).drop(columns=['id'])
data_tag = pd.read_csv('../../preprocess_data/train_x_33465.csv',usecols=['tag'])

data = pd.concat([data_date,data_raw,data_null,data_tag],axis=1)
data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])

x = data.fillna(-1).values
y = data_label.values.ravel()

CPU times: user 1min 5s, sys: 4.9 s, total: 1min 10s
Wall time: 1min 17s


使用旧数据集 train集的维度为：  
使用新数据集 train集的维度为：6702

### 读取无标签的

In [5]:
%%time
unlabel_date = pd.read_csv('../../preprocess_data/unlabel_x_date.csv').drop(columns=['id','loan_hour'])
unlabel_raw = pd.read_csv('../../preprocess_data_new/train_ax_nodup.csv',skiprows=range(1,33466)).drop(columns=['id','loan_dt','tag'])
unlabel_null = pd.read_csv('../../preprocess_data_new/train_ax_row_null.csv',skiprows=range(1,33466)).drop(columns=['id'])
unlabel_tag = pd.read_csv('../../preprocess_data/unlabel_x_66535.csv',usecols=['tag'])
unlabel = pd.concat([unlabel_date,unlabel_raw,unlabel_null,unlabel_tag],axis=1)
x_test = unlabel.fillna(-1).values
label = pd.read_csv('../predict_unlabel/preds/unlabel_pred.csv',usecols=['prob'])
label.columns = ['label']

CPU times: user 2min 16s, sys: 20.7 s, total: 2min 36s
Wall time: 2min 47s


In [6]:
low,high = np.percentile(label.label,5),np.percentile(label.label,95)
myfilter = (label.label<low)|(label.label>high)

unlabel_f = unlabel[myfilter]
label_f = label[myfilter].copy()

label_f[label_f<low] = 0
label_f[label_f>high] = 1

label_f.label.value_counts()

1.0    3327
0.0    3327
Name: label, dtype: int64

### 合并数据

In [7]:
data_com = pd.concat([data,unlabel_f],copy=False,axis=0,ignore_index=True)
label_com = pd.concat([data_label,label_f],copy=False,axis=0,ignore_index=True)

In [8]:
x_train,y_train= data_com.fillna(-1).values, label_com.values.ravel()

### 定义学习器

In [9]:
def SelectModel(model_name):
    if model_name == 'GBC':
        from sklearn.ensemble import GradientBoostingClassifier
        model = GradientBoostingClassifier(loss='deviance',
                                           learning_rate =0.1,
                                           n_estimators=300,
                                           subsample=0.9,
                                           max_depth=3,
#                                            verbose=1,
                                          random_state=2018)
    elif model_name == 'XGB':
        from xgboost import XGBClassifier

        model = XGBClassifier(max_depth=6,
                              learning_rate =0.04, 
                              booster='gbtree',
                              objective='binary:logistic',
                              early_stopping_rounds=100,
                              scale_pos_weight=float(len(y_train)-np.sum(y_train))/float(np.sum(y_train)),
                              eval_metric='auc',
                              gamma=1,
                              reg_lambda=1,
                              subsample=0.9,
                              min_child_weight=1,
                              seed=2018,
                              silent=False,
                              n_jobs=24,
                              num_boost_round =300
                             )
    elif model_name == 'RFC':
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier(n_estimators=1500,
                                       n_jobs =36,
                                       max_features='sqrt',
                                       class_weight='balanced',
#                                        verbose =1,
                                       random_state=2018)
    elif model_name == 'LGB':
        from lightgbm import LGBMClassifier
        model = LGBMClassifier(boost='gbdt',
                    num_leaves=135, 
                    scale_pos_weight=float(len(y_train)-np.sum(y_train.ravel()))/float(np.sum(y_train.ravel())),
                    max_depth=-1,
                    learning_rate=.05,
                    max_bin=200,
                    min_data_in_leaf= 60,
                    objective='binary',
                    metric='auc',
                    num_threads=32,
                    slient=False,
                    num_boost_round =300)
    else:
        pass
    return model

## 训练

In [ ]:
%%time
import joblib
model_list = ['XGB','RFC','LGB']
unlabel_pred_list = []
path = './model_5_95'
if not os.path.exists(path):
    os.mkdir(path)

for model in model_list:
    print(model)
    clf = SelectModel(model)
    clf.fit(x_train,y_train)
    joblib.dump(clf,os.path.join(path,model))  

XGB


## 预测

In [55]:
%%time
valid_date = pd.read_csv('../../preprocess_data/valid_date.csv').drop(columns=['id','loan_hour'])
# valid_raw = pd.read_csv('../../preprocess_data/valid.csv').drop(columns=['id','loan_dt'])
valid_raw = pd.read_csv('../../preprocess_data_new/valid_nodup.csv').drop(columns=['id','loan_dt'])
valid_null = pd.read_csv('../../preprocess_data_new/valid_row_null.csv').drop(columns=['id'])
valid_tag = pd.read_csv('../predict_tag/tag.csv',usecols=['tag'])

CPU times: user 31.9 s, sys: 1.29 s, total: 33.2 s
Wall time: 33.2 s


In [56]:
%%time
valid = pd.concat([valid_date,valid_raw,valid_null,valid_tag],axis=1)

CPU times: user 1.32 s, sys: 4 ms, total: 1.33 s
Wall time: 1.32 s


In [57]:
test_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values

In [116]:

dtest = xgb.DMatrix(valid.fillna(-1).values, feature_names=list(valid.columns))

raw 去除nan列 + 统计null AUC: 0.82751113123698  
nodup + null + tag AUC: 0.82803008109167  
nodup + null + tag（rank融合）AUC:0.8279914450872  
nodup + null + tag (fillna(-1)) AUC:0.82979480823375

In [27]:
data.shape

(33465, 6702)